In [3]:
# Import dependencies
import pandas as pd
import psycopg2
from config import DB_HOST, DB_USER, DB_PASS, DB_PORT
from sqlalchemy import create_engine

In [3]:
# Import csv files into dataframes
emissions_df = pd.read_csv('../Resources/US_Historical_Clean_All_Gases.csv', index_col = 0)
gdp_df = pd.read_csv('../Resources/US_GDPbyState_Cleaned.csv', index_col = 0)
pop_df = pd.read_csv('../Resources/US_PopbyState_Cleaned.csv', index_col = 0)

In [6]:
gdp_pop_sector_emissions = pd.read_csv('../MergedTables/gdp_pop_sector_emissions.csv')

In [4]:
#Create emissions_state_df
emissions_state_df = emissions_df.groupby(['Country','State', 'Year'], as_index = False)['All GHG', 'CH4', 'CO2', 'N2O', 'F-Gas'].apply(lambda x : x.astype(float).sum())
emissions_state_df

C:\ProgramData\Anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


Country State  Year     All GHG        CH4        CO2       N2O  \
0         US    AK  1990   90.824201  16.256715  84.206963  3.954116   
1         US    AK  1991   92.287600  16.093682  87.346097  4.159046   
2         US    AK  1992   94.953720  15.907576  84.772492  4.350361   
3         US    AK  1993   95.270389  15.854616  85.109545  4.664578   
4         US    AK  1994   93.447679  14.584147  82.843112  4.641887   
...      ...   ...   ...         ...        ...        ...       ...   
1503      US    WY  2014  210.555235   0.000000   0.000000  0.000000   
1504      US    WY  2015  180.691787   0.000000   0.000000  0.000000   
1505      US    WY  2016  162.389203   0.000000   0.000000  0.000000   
1506      US    WY  2017  198.608349   0.000000   0.000000  0.000000   
1507      US    WY  2018  179.014581   0.000000   0.000000  0.000000   

          F-Gas  
0     18.787770  
1     20.527548  
2     22.396932  
3     24.441791  
4     26.063689  
...         ...  
1503   0.000000  
1504   0.000000  
1505   0.000000  
1506   0.000000  
1507   0.000000  

[1508 rows x 8 columns]

In [5]:
#Change gdp_df column names to match other tables
gdp_df.rename(columns = {'State':'StateFull', 'Abbrev':'State'}, inplace = True)
gdp_df

StateFull State  Year           GDP
0     United States   USA  1997  3.286530e+12
1     United States   USA  1998  3.384010e+12
2     United States   USA  1999  3.592860e+12
3     United States   USA  2000  3.733600e+12
4     United States   USA  2001  3.866600e+12
...             ...   ...   ...           ...
1243        Wyoming    WY  2016  1.516490e+10
1244        Wyoming    WY  2017  1.597636e+10
1245        Wyoming    WY  2018  1.712228e+10
1246        Wyoming    WY  2019  1.697884e+10
1247        Wyoming    WY  2020  1.546964e+10

[1248 rows x 4 columns]

In [6]:
# Remove decimal from Year in pop_df
pop_df['Year'] = pop_df['Year'].astype(int)

In [7]:
# Add total US Population to Pop DF
total_pop = pop_df.groupby(pop_df['Year']).sum()['Population']
total_pop = pd.DataFrame(total_pop).reset_index()
total_pop = total_pop.reindex(columns = ['State', 'Year', 'Population'])
pop_df_with_total = pop_df.append(total_pop)
pop_df_with_total.fillna('United States', inplace = True)
pop_df_with_total = pop_df_with_total.reset_index(drop = True)
pop_df_with_total = pop_df_with_total.rename(columns = {"State":"StateFull"})
pop_df_with_total

StateFull  Year  Population
0           Alabama  1990     4050055
1           Alabama  1991     4099156
2           Alabama  1992     4154014
3           Alabama  1993     4214202
4           Alabama  1994     4260229
...             ...   ...         ...
1607  United States  2016   323071755
1608  United States  2017   325122128
1609  United States  2018   326838199
1610  United States  2019   328329953
1611  United States  2020   329484123

[1612 rows x 3 columns]

In [4]:
#establishing the connection
conn = psycopg2.connect(
   database="postgres", user=DB_USER, password=DB_PASS, host=DB_HOST, port= DB_PORT
)
conn.autocommit = True


In [14]:
# Create Table Function
def create_db_table(db, table_name):
    
    # Connect to SQL database for loading
    db_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:5432/us_emissions_db"
    
    # Create Engine
    engine = create_engine(db_string)
    
    # Load table
    db.to_sql(name= table_name, con=engine, if_exists = 'replace')
      

In [10]:
# Load US Emissions by State By Sector
create_db_table(emissions_df, "sector_emissions")

In [11]:
# Load US Emissions by State
create_db_table(emissions_state_df, "state_emissions")

In [12]:
# Load Population Data
create_db_table(pop_df_with_total, "state_pop")

In [13]:
# Load GDP Data
create_db_table(gdp_df, "state_gdp")

In [15]:
# Load combined master table
create_db_table(gdp_pop_sector_emissions, "gdp_pop_sector_emissions")